# Contents
* [Introduction](#Introduction)
* [Installation](#Installation)
* [Prover](#Prover)
    * [Prover Class]()
    * [Custom Constructor]()
    * [Single-Task Prover]()
* [Selection Strategies](#Selection-Strategies)
    * [Predictions Class](#Attributes)
    * [Goal Embedding](#Goal-Embedding)
    * [Theorem Embedding](#Theorem-Embedding)
    * [Proof State Search](#Proof-State)
    * [Proof State Embedding](#Proof-State-Embedding)
    * [Proof State Encoding](#Proof-State-Encoding)
    * [Tactic Selection](#Tactic-Selection)
    * [Premise Selection](#Premise-Selection)
* [Related Work](#Related-Work)  

[Run](#Run-DeepHol)

---
---
---

# Introduction  

## How to Use This Notebook  
First, follow the instructions below to install deephol in the same directory as this notebook.  

By reading through the rest of the documentation, you will be able to implement your own theorem prover via this notebook.  

Within this documentation are code cells where you can implement your own functions. Once you write the function, run the cell to overwrite it. You can also choose to leave them blank and deephol will fall back on default behavior.  

Use the [cell](#Run-DeepHol) at the bottom to run DeepHOL.

# Installation
## Basic Dependencies
* Anaconda Python
* gcc
* g++
* Docker


# Building from source
Follow this guide if you want to build on your local machine without using the container.

## Setting up your virtual environment
Create and activate a conda environment.

In [ ]:
%%bash
conda create --name deephol
conda activate deephol
pip install h5py six numpy scipy wheel mock pyfarmhash grpcio
pip install keras_applications==1.0.6 keras_preprocessing==1.0.5 --no-deps

## Clone the deepmath repository

In [ ]:
#TODO: Update this Repository to include the HolJup files
!git clone https://github.com/tensorflow/deepmath.git

## Install Bazel
Bazel is a build manager for tensorflow. We need to install this in order to build deepmath.

In [ ]:
%%bash
wget https://github.com/bazelbuild/bazel/releases/download/0.21.0/bazel-0.21.0-installer-linux-x86_64.sh
chmod 777 bazel-0.21.0-installer-linux-x86_64.sh
bash bazel-0.21.0-installer-linux-x86_64.sh --prefix=$HOME/bazel
PATH=$HOME/bazel/bin:$PATH

### Configure tensorflow
Navigate to the tensorflow dirctory and configure the following options. You can make changes here depending on your system.

In [ ]:
%%bash
cd deepmath/tensorflow
TF_IGNORE_MAX_BAZEL_VERSION=1   TF_NEED_OPENCL_SYCL=0   TF_NEED_COMPUTECPP=1   TF_NEED_ROCM=0   TF_NEED_CUDA=0   TF_ENABLE_XLA=0   TF_DOWNLOAD_CLANG=0   TF_NEED_MPI=0   TF_SET_ANDROID_WORKSPACE=0 CC_OPT_FLAGS="-march=native -Wno-sign-compare"  ./configure

Tensorflow will ask you to specify a python path. It will also give you a default option. Select the default and take note of the path it gives you.  
Now set the environment variable to that value.

In [ ]:
%%bash
export $PYTHON_BIN_PATH=<path given by tensorflow>

## Build tensorflow with Bazel

This step will take awhile. go fix yourself some snacks while it works.

In [ ]:
%%bash
cd ..
bazel build -c opt //deepmath/deephol:main --define grpc_no_ares=true --python_path=$PYTHON_BIN_PATH

## Run deephol
### Download and Setup HOL-Light container
Create a docker network with

In [ ]:
!docker network create holist_net

Now run the HOL-Light image

In [ ]:
!docker run -d --network=holist_net --name=holist gcr.io/deepmath/hol-light

Now you can get the ip of the HOL-Light image with

In [ ]:
!docker network inspect holist_net

Now you can run deephol with

In [ ]:
python bazel-bin/deepmath/deephol/main --prover_options=data/configuration/prover_options.textpb --output=proof_logs.textpbs --proof_assistant_server_address=<ip of proof assistant>

## Verify using the proof checker
//TODO

In [ ]:
//TODO

---
---
---
# Prover
//TODO  
By default, deephol will use the BFS prover defined in `prover.py`. You can use these methods to implement your own

## Custom Constructor
If your prover needs a custom `__init__()`, you can implement it here.  

*note that the function in this file is called `_init_` rather than `__init__`. This is an intentional implementation detail and not a typo*

In [ ]:
 %%writefile deepmath/deephol/jup_prover/custom/constructor.py

###### your code below #####
    
def _init_(self, prover_options: deephol_pb2.ProverOptions, hol_wrapper,
               action_gen: action_generator.ActionGenerator,
               theorem_db: proof_assistant_pb2.TheoremDatabase):
    super(NoBacktrackProver, self).__init__(
        prover_options, hol_wrapper, theorem_db, single_goal=True)
    self.action_gen = action_gen

## Prove One
This is the only method that needs to be implemented. It should search the proof tree to find a single proof, and return `None` upon success, or an error message upon failure.

In [ ]:
def prove_one(self, tree: proof_search_tree.ProofSearchTree,
                task: proof_assistant_pb2.ProverTask) -> Optional[Text]:
    """Searches for a proof via BFS.
    Args:
      tree: Search tree with a single goal node to be proved.
      task: ProverTask to be performed.
    Returns:
      None on success and error message on failure.
    """
    root = tree.nodes[0]
    nodes_explored = 0
    # Note that adding new node to the tree might re-enable previous nodes
    # for tactic applications, if they were marked to be ignored by
    # failing sibling nodes.
    tree.cur_index = 0
    while not self.timed_out() and not root.closed and not root.failed and (
        nodes_explored < self.options.max_explored_nodes):
      if tree.cur_index >= len(tree.nodes):
        return 'BFS: All nodes are failed or ignored.'
      node = tree.nodes[tree.cur_index]
      tree.cur_index += 1
      if node.ignore or node.failed or node.closed or node.processed:
        continue
      nodes_explored += 1
      # Note that the following function might change tree.cur_index
      # (if a node that was ignored suddenly becomes subgoal of a new
      # tactic application).
      prover_util.try_tactics(node, self.options.max_top_suggestions,
                              self.options.min_successful_branches,
                              self.options.max_successful_branches,
                              task.premise_set, self.action_gen,
                              self.prover_options.tactic_timeout_ms)
    root_status = ' '.join([
        p[0] for p in [('closed', root.closed), ('failed', root.failed)] if p[1]
    ])
    tf.logging.info('Timeout: %s root status: %s explored: %d',
                    str(self.timed_out()), root_status, nodes_explored)
    if self.timed_out():
      return 'BFS: Timeout.'
    elif root.failed:
      return 'BFS: Root Failed.'
    elif nodes_explored >= self.options.max_explored_nodes and not root.closed:
      return 'BFS: Node limit reached.'

# Selection Strategies

## Attributes
A few words about the class attributes that don't need to be implemented.


## Goal Embedding 

**`_batch_goal_embedding(self, goals: List[Text]) -> BATCH_GOAL_EMB_TYPE`**  
Compute embeddings from a list of goals   
  
  

#### Parameters:
-   `goals: List[Text]`
List of goals as strings

#### Return Value:
-   `goal_embeddings: numpy.ndarray`  



In [2]:
%%writefile deepmath/deephol/jup_predict/custom/goal_emb.py

###### your code below #####

import numpy as np

def _batch_goal_embedding(predictor, goals) -> BATCH_GOAL_EMB_TYPE:
    return np.array([[goal.__hash__(), 0] for goal in goals])

## Theorem Embedding 

**` _batch_thm_embedding(self, thms: List[Text]) -> BATCH_THM_EMB_TYPE`**  
From a list of string theorems, computes and returns their embeddings.  

#### Parameters
-   `thms: List[Text]`  
List of theorems as strings

#### Return Value
- `theorem_embeddings: numpy.ndarray`

In [ ]:
%%writefile deepmath/deephol/jup_predict/custom/thm_emb.py

###### your code below #####

import numpy as np

def _batch_thm_embedding(predictor, thms):
    """From a list of string theorems, compute and return their embeddings."""
    return np.array([[thm.__hash__(), 1] for thm in thms])

## Proof State 

### `proof_state_from_search(self, node: proof_search_tree.ProofSearchNode) -> ProofState:`
Convert from `proof_search_tree.ProofSearchNode` to proof state.

#### Parameters
-   `node: proof_search_tree.ProofSearchNode`

#### Return Value
-   `proof_state: predictions.ProofState`

In [ ]:
%%writefile deepmath/deephol/jup_predict/custom/st_search.py

###### your code below #####

from deepmath.deephol import predictions

def _proof_state_from_search(predictor, node):
    return predictions.ProofState(goal='goal')

## Proof State Embedding 

### `proof_state_embedding(self, state: ProofState) -> EmbProofState:`
From a proof state, computes and returns embeddings of each component.  

#### Parameters
-   `state: predictions.ProofState`

#### Return Value
-   `proof_state_embedding: predictions.EmbProofState`


In [ ]:
%%writefile deepmath/deephol/jup_predict/custom/st_emb.py

###### your code below #####

from deepmath.deephol import predictions

def _proof_state_embedding(predictor, state):
    return predictions.EmbProofState(*[[x.__hash__(), 2] for x in state])

## Proof State Encoding 

### `proof_state_encoding(self, state_emb: EmbProofState) -> STATE_ENC_TYPE:`
From an embedding of a proof state, computes and returns its encoding.

#### Parameters
-   `state_emb: predictions.EmbProofState`

#### Return Value
-   `state_encoding: numpy.ndarray`


In [ ]:
%%writefile deepmath/deephol/jup_predict/custom/st_enc.py

###### your code below #####

def _proof_state_encoding(predictor, state_emb):
    return state_emb.goal_emb

## Tactic Selection

### `_batch_tactic_scores(self, state_encodings: List[STATE_ENC_TYPE]) -> np.ndarray:`
Predicts tactic probabilities for a batch of goals.  

#### Parameters
-   `state_encodings: List[numpy.ndarray[TODO]]`

#### Return Value
-   `tactic_scores: numpy.ndarray[batch_size, num_tactics]`  

2D array of dimension \[batch_size, num_tactics\] representing tactic probabilities. Batch size is the length of `goal_embeddings`

In [2]:
%%writefile deepmath/deephol/jup_predict/custom/tac_sc.py

###### your code below #####

import numpy as np

def _batch_tactic_scores(predictor, goal_embeddings):
    num_tactics = 41
    return np.array([[np.sum(emb)] + [.0]*40 for emb in goal_embeddings]) 

Overwriting deepmath/deephol/jup_predict/custom/tac_sc.py


## Premise Selection

### ` _batch_thm_scores(self, state_encodings: List[STATE_ENC_TYPE], thm_embeddings: BATCH_THM_EMB_TYPE, tactic_id: Optional[int] = None) -> List[float]:` 
Predict relevance scores for goal, theorem pairs.  

#### Parameters
-   `state_encodings: List[numpy.ndarray]`
-   `theorem_embeddings: numpy.ndarray`
-   `tactic_id: int` (optional)

#### Return Value
-   `tactic_scores: List[float]`

In [ ]:
%%writefile deepmath/deephol/jup_predict/custom/tac_sc.py

###### your code below #####

import numpy as np

def _batch_thm_scores(predictor, goal_embeddings, thm_embeddings, tactic_id=None):
    if tactic_id is not None:
        c = 3.0 * float(tactic_id)
    else:
        c = 2.0
    return np.array([
          np.sum(e1) + c * np.sum(e2)
          for (e1, e2) in zip(goal_embeddings, thm_embeddings)
  ])

---


---
---
---

# Run DeepHol

In [ ]:
!bash run.sh

# Related Work

## Papers
* [HOList: An Environment for Machine Learning of Higher-Order Theorem Proving](https://arxiv.org/pdf/1904.03241.pdf)  
* [Learning to Prove Theorems via Interacting with Proof Assistants](https://arxiv.org/pdf/1905.09381.pdf)  

## Resources 
* [HOList Site](https://sites.google.com/view/holist/home)
* [Deepmath Repository](https://github.com/tensorflow/deepmath) 
* [HOL-Light Repository](https://github.com/brain-research/hol-light)
* [CoqGym](https://github.com/princeton-vl/CoqGym)



# About This Project
This tutorial was made as part of an undergraduate research project at University of Central Florida, supervised by Dr. Sumit Jha.

# Contact
* Aaron Hadley: aahadley1@gmail.com
* Tyler McFadden: neruelin@gmail.com
* Jordan Starkey: jordanstarkey95@gmail.com
* Sumit Jha: Sumit.Jha@ucf.edu